# Comvolution / Pooling レイヤの実装

## 4次元配列

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'sample')))
import numpy as np

In [2]:
x = np.random.rand(10, 1, 28, 28) # ランダムにデータを生成
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape # 1つ目のデータにアクセス

(1, 28, 28)

In [4]:
x[1].shape # 2つ目のデータにアクセス

(1, 28, 28)

In [5]:
x[0, 0] # 1つ目のデータの、1チャンネル目の空間データにアクセス

array([[4.68702643e-01, 8.25601581e-01, 8.00818513e-01, 9.07497207e-01,
        3.84838594e-01, 2.58556252e-01, 3.39597484e-02, 4.30849315e-01,
        7.27769576e-02, 1.50412707e-01, 6.34354502e-01, 4.84647852e-01,
        8.55548964e-01, 6.92284364e-01, 1.11238541e-01, 3.33037680e-01,
        3.81357683e-01, 9.20544113e-01, 5.65142521e-01, 9.98015298e-01,
        9.91598490e-01, 8.85438293e-01, 2.68954365e-01, 3.70784565e-01,
        8.87733992e-01, 8.05377060e-02, 8.61421901e-01, 1.78534870e-01],
       [4.14100558e-01, 1.19371526e-01, 2.60757876e-01, 1.09816633e-02,
        1.10734004e-01, 8.53063695e-01, 9.33674314e-01, 5.11410455e-01,
        5.50644555e-01, 7.98962949e-01, 4.66828370e-01, 1.56990324e-01,
        3.97926893e-01, 6.49015005e-02, 9.38890296e-01, 7.81818732e-01,
        9.98471239e-01, 7.70890935e-01, 2.13693729e-01, 8.75135715e-01,
        7.12718992e-01, 8.58193749e-01, 7.26171379e-01, 4.36085969e-02,
        1.98453242e-01, 7.54876109e-01, 5.17852647e-01, 7.60726

## im2colによる展開
- NumPyでは、`for`分を使うと処理が遅くなってしまう
- `for`分を使わず、`im2col`を使う

`im2col`は image to columnの略、「画像から行列へ」

## Convolution レイヤの実装

In [6]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'sample')))
import numpy as np
from common.util import im2col

# パッチサイズが1, チャンネル3の 7x7のデータ
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(f'col1.shape: {col1.shape}')

# パッチサイズが10, チャンネル3の 7x7のデータ
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(f'col2.shape: {col2.shape}')

col1.shape: (9, 75)
col2.shape: (90, 75)


In [7]:
class Comvolution:
    """
    畳み込み層
    """
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        """
        順伝搬
        """
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad) # 入力データの展開
        col_W = self.W.reshape(FN, -1).T               # フィルターを2次元に展開
        out = np.dot(col, col_W) + self.b              # 展開した行列の籍を計算
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        """
        逆伝搬
        """
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

## Poolingレイヤの実装

In [8]:
class Pooling:
    """
    プーリング層
    """
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        """
        順伝搬
        """
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 入力データの展開
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 行ごとに最大値をとる
        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        
        # 適切な出力サイズに整形する
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        """
        逆伝搬
        """
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx